In [22]:
# Import Dependancies
import pandas as pd
from path import Path
import sklearn as skl
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [23]:
# Load data
file_path = "../diabetes-filtered.csv"
df2 = pd.read_csv(file_path)
df2.head()

,id,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,metformin,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,change,diabetesMed,readmitted
0,1,Caucasian,Female,5,6,25,1,1,41,0,...,0,0,0,0,0,0,0,0,0,NO
1,2,Caucasian,Female,15,1,1,7,3,59,0,...,0,0,0,0,0,0,3,1,1,>30
2,3,AfricanAmerican,Female,25,1,1,7,2,11,5,...,0,0,2,0,0,0,0,0,1,NO
3,4,Caucasian,Male,35,1,1,7,2,44,1,...,0,0,0,0,0,0,3,1,1,NO
4,5,Caucasian,Male,45,1,1,7,1,51,0,...,0,0,2,0,0,0,2,1,1,NO


In [24]:
#Encode Reamditted Column

df2['readmitted']=df2['readmitted'].replace('NO', 0)
df2['readmitted']=df2['readmitted'].replace('>30', 1)
df2['readmitted']=df2['readmitted'].replace('<30', 1)

# select the best features 
df2=df2[['num_lab_procedures','num_medications', 'time_in_hospital', 'diag_2', 'diag_3', 'diag_1', 'age', 'number_diagnoses', 'num_procedures','discharge_disposition_id', 'admission_type_id',
'insulin', 'admission_source_id', 'number_inpatient', 'A1Cresult', 'metformin', 'number_outpatient', 'gender', 'race','readmitted']]

In [25]:
#Encoding the Data
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse = False)

#Encode Column Race
encode_df = pd.DataFrame(enc.fit_transform(df2.race.values.reshape(-1,1)))

#Rename encoded columns
encode_df.columns = enc.get_feature_names(['race'])
encode_df.head()

# Merge Dataframes
df2 = df2.merge(encode_df,left_index=True,right_index=True)

#Encoding the Data
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse = False)

#Encode Column Race
encode_df2 = pd.DataFrame(enc.fit_transform(df2.gender.values.reshape(-1,1)))

#Rename encoded columns
encode_df2.columns = enc.get_feature_names(['gender'])

# Merge Dataframes
df2 = df2.merge(encode_df2,left_index=True,right_index=True)

c:\Users\Rahul\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\Rahul\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
# Drop Encoded Columns and ID Column

df2.drop(columns = ['race','gender',], inplace = True)

In [27]:
# Classify X and y Data

y = df2['readmitted']
X = df2.drop(columns = ['readmitted'])

In [28]:
# Create testing and training set

X_train,X_test,y_train,y_test = train_test_split(X,y, train_size = 0.8, random_state = 6, stratify = y)

In [29]:
# Resample the training data with the RandomOversampler

from collections import Counter
Counter(y_train)
from imblearn.over_sampling import RandomOverSampler
sampler = RandomOverSampler(random_state =4)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 34388, 1: 34388})

In [30]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_resampled)

# Scale the data
X_train_scaled = X_scaler.transform(X_resampled)
X_test_scaled = X_scaler.transform(X_test)

Randon Forest Classifier

In [31]:
# Create a random forest classifier.
model = RandomForestClassifier(n_estimators=128, random_state=6) 
# Fitting the model.
model = model.fit(X_train_scaled, y_resampled)

In [32]:
# Making predictions using the testing data.
y_pred = model.predict(X_test_scaled)

In [33]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.6211549217002237

In [34]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual NO", "Actual Readmitted"], columns=["Predicted NO", "Predicted Readmitted"])

cm_df

,Predicted NO,Predicted Readmitted
Actual NO,6602,1995
Actual Readmitted,3424,2283


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.77      0.71      8597
           1       0.53      0.40      0.46      5707

    accuracy                           0.62     14304
   macro avg       0.60      0.58      0.58     14304
weighted avg       0.61      0.62      0.61     14304



EasyEnsemble Classifier

In [36]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
model1 = EasyEnsembleClassifier(random_state= 6)

model1.fit(X_train_scaled,y_resampled)

EasyEnsembleClassifier(random_state=6)

In [38]:
#Obtaining predicted values
y_pred1 = model1.predict(X_test_scaled)

# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred1)

0.5963940828091935

In [39]:
# Display the confusion matrix

from sklearn.metrics import confusion_matrix, classification_report
cm1 = confusion_matrix(y_test, y_pred1)

# Create a DataFrame from the confusion matrix.
cm_df1 = pd.DataFrame(
    cm1, index=["Actual NO", "Actual Readmitted"], columns=["Predicted NO", "Predicted Readmitted"])

cm_df1

,Predicted NO,Predicted Readmitted
Actual NO,5214,3383
Actual Readmitted,2361,3346


In [40]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred1))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.69      0.61      0.59      0.64      0.60      0.36      8597
          1       0.50      0.59      0.61      0.54      0.60      0.35      5707

avg / total       0.61      0.60      0.59      0.60      0.60      0.36     14304



In [ ]:
# Calculate feature importance in the Random Forest model.
importances = model.feature_importances_
importances

array([1.32059055e-01, 1.09823521e-01, 7.34232194e-02, 7.51606680e-02,
       7.30454374e-02, 7.26830708e-02, 6.00692465e-02, 5.28819985e-02,
       5.06502867e-02, 5.01908095e-02, 3.13123848e-02, 4.02751895e-02,
       2.48313744e-02, 2.20204129e-02, 2.34378077e-02, 2.25327740e-02,
       1.69384571e-02, 1.18564973e-02, 1.68950442e-03, 1.33823974e-02,
       3.60420690e-03, 5.49168187e-03, 1.63835673e-02, 1.62537662e-02,
       2.66480504e-06])

In [ ]:
# We can sort the features by their importance.
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.13205905523951394, 'num_lab_procedures'),
 (0.10982352135776836, 'num_medications'),
 (0.07516066800494298, 'diag_2'),
 (0.07342321940034278, 'time_in_hospital'),
 (0.07304543744712055, 'diag_3'),
 (0.07268307079088487, 'diag_1'),
 (0.06006924647795027, 'age'),
 (0.05288199846777144, 'number_diagnoses'),
 (0.0506502867488169, 'num_procedures'),
 (0.05019080946066101, 'discharge_disposition_id'),
 (0.04027518946361675, 'insulin'),
 (0.031312384794661155, 'admission_type_id'),
 (0.024831374439491943, 'admission_source_id'),
 (0.023437807715108276, 'A1Cresult'),
 (0.022532774038474177, 'metformin'),
 (0.02202041289396333, 'number_inpatient'),
 (0.016938457139683852, 'number_outpatient'),
 (0.016383567281215445, 'gender_Female'),
 (0.016253766222894095, 'gender_Male'),
 (0.013382397362566795, 'race_Caucasian'),
 (0.011856497258657096, 'race_AfricanAmerican'),
 (0.005491681869311818, 'race_Other'),
 (0.003604206902146285, 'race_Hispanic'),
 (0.0016895044173991893, 'race_Asian'),
 (2.664